In [22]:
import pandas as pd
import subprocess
import json
import sys

pd.set_option('max_colwidth',250)
pd.set_option('display.max_rows',90)
# sys.setdefaultencoding('utf-8')

AttributeError: module 'sys' has no attribute 'setdefaultencoding'

Explorando o Catalogo

In [2]:
cat = pd.read_csv("catalogo_de_dados.csv", on_bad_lines='skip',sep=";")

In [3]:
categoriasSenadores = cat.query("nom_sub_categoria == 'Senadores'")

In [4]:
senadoresExercicio = categoriasSenadores.query("des_grupo_dados == 'Senadores em exercício'")

In [5]:
conjSenadoresExercicio = senadoresExercicio.query("nom_conjunto_dados == 'Senadores em exercício' ")

Capturando Parlamentares em Exercicio

In [51]:
comando = 'curl https://legis.senado.leg.br/dadosabertos/arquivos/ListaParlamentarEmExercicio.json "Content-Type: text/xml; charset=utf-8-sig"'
resultado = subprocess.run(comando, shell=True, text=True, capture_output=True)
dados = json.loads(resultado.stdout)
dados = pd.DataFrame(dados)
dados = (dados.loc["Parlamentares"]["ListaParlamentarEmExercicio"])
dados = pd.DataFrame(dados["Parlamentar"])
saida = pd.DataFrame()
for index,row in dados.iterrows():
    IdentificacaoParlamentar = pd.DataFrame(row["IdentificacaoParlamentar"])
    Mandato = pd.DataFrame(row["Mandato"])
    concatenar = pd.concat([IdentificacaoParlamentar,Mandato])
    saida = pd.concat([saida,concatenar])
# dados = pd.json_normalize(dados)
# dados = resultado.stdout
# dados = pd.read_json(dados, orient='index')

In [52]:
IdentificacaoParlamentar = dados["IdentificacaoParlamentar"]
Mandato = dados["Mandato"]

In [53]:
dadosTMP = pd.DataFrame() 
for i in range(0,81):
    tmpIP = pd.DataFrame(IdentificacaoParlamentar[i])
#     mpM = pd.DataFrame(Mandato[i])
    tmpIP = tmpIP.reset_index()
#     mpM = tmpM.reset_index()
    tmpTel = tmpIP.iloc[0]["Telefones"]
    try:
        tmpTels = pd.DataFrame(tmpTel, index=["NumeroTelefone","OrdemPublicacao","IndicadorFax"])
    except:
        tmpTels = pd.DataFrame(tmpTel[0], index=["NumeroTelefone","OrdemPublicacao","IndicadorFax"])
        
    tmpIP.loc[0, "Telefones"] = tmpTels.loc["NumeroTelefone", "NumeroTelefone"]
    tmpIP.loc[0, "CodigoBloco"] = tmpIP.loc[1, "Bloco"]
    tmpIP.loc[0, "NomeBloco"] = tmpIP.loc[2, "Bloco"]
    tmpIP.loc[0, "NomeApelido"] = tmpIP.loc[3, "Bloco"]
    tmpIP.loc[0, "DataCriacao"] = tmpIP.loc[4, "Bloco"]
    tmpIP.drop([1,2,3,4], inplace=True)
    #mpIP.join(tmpM)
    dadosTMP = pd.concat([tmpIP,dadosTMP])
    dados = dadosTMP.drop(columns="index").reset_index().drop(columns="index")

C:\Users\kristyan.silva\AppData\Local\Temp\ipykernel_16028\2143137496.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '336' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tmpIP.loc[0, "CodigoBloco"] = tmpIP.loc[1, "Bloco"]
C:\Users\kristyan.silva\AppData\Local\Temp\ipykernel_16028\2143137496.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Bloco Parlamentar Democracia' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tmpIP.loc[0, "NomeBloco"] = tmpIP.loc[2, "Bloco"]
C:\Users\kristyan.silva\AppData\Local\Temp\ipykernel_16028\2143137496.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'BLDEM' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.


In [54]:
dados

,CodigoParlamentar,CodigoPublicoNaLegAtual,NomeParlamentar,NomeCompletoParlamentar,SexoParlamentar,FormaTratamento,UrlFotoParlamentar,UrlPaginaParlamentar,EmailParlamentar,Telefones,SiglaPartidoParlamentar,UfParlamentar,Bloco,MembroMesa,MembroLideranca,CodigoBloco,NomeBloco,NomeApelido,DataCriacao,UrlPaginaParticular
0,3806,836,Zequinha Marinho,JosÃ© da Cruz Marinho,Masculino,Senador,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador3806.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/3806,sen.zequinhamarinho@senado.leg.br,33036623,PODEMOS,PA,NaN,NÃ£o,NÃ£o,336,Bloco Parlamentar Democracia,BLDEM,2023-02-02,NaN
1,5783,818,Zenaide Maia,Zenaide Maia Calado Pereira dos Santos,Feminino,Senadora,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador5783.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/5783,sen.zenaidemaia@senado.leg.br,33032371,PSD,RN,NaN,NÃ£o,Sim,335,Bloco Parlamentar da ResistÃªncia DemocrÃ¡tica,BLPRD,2023-02-01,NaN
2,5070,870,Wilder Morais,Wilder Pedro de Morais,Masculino,Senador,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador5070.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/5070,sen.wildermorais@senado.leg.br,33036440,PL,GO,NaN,NÃ£o,NÃ£o,338,Bloco Parlamentar Vanguarda,BLVANG,2023-02-06,https://www.wildermorais.com.br/
3,5411,808,Weverton,Weverton Rocha Marques de Sousa,Masculino,Senador,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador5411.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/5411,sen.wevertonrocha@senado.leg.br,33034161,PDT,MA,NaN,Sim,Sim,336,Bloco Parlamentar Democracia,BLDEM,2023-02-02,NaN
4,5748,821,Veneziano Vital do RÃªgo,Veneziano Vital do RÃªgo Segundo Neto,Masculino,Senador,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador5748.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/5748,sen.venezianovitaldorego@senado.leg.br,33032252,MDB,PB,NaN,Sim,NÃ£o,336,Bloco Parlamentar Democracia,BLDEM,2023-02-02,NaN
5,5899,864,Vanderlan Cardoso,Vanderlan Vieira Cardoso,Masculino,Senador,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador5899.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/5899,sen.vanderlancardoso@senado.leg.br,33032092,PSD,GO,NaN,NÃ£o,NÃ£o,335,Bloco Parlamentar da ResistÃªncia DemocrÃ¡tica,BLPRD,2023-02-01,NaN
6,5736,876,Tereza Cristina,Tereza Cristina Correa da Costa Dias,Feminino,Senadora,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador5736.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/5736,sen.terezacristina@senado.leg.br,33032431,PP,MS,NaN,NÃ£o,Sim,346,Bloco Parlamentar AlianÃ§a,BLALIANÃ‡A,2023-03-20,NaN
7,6338,879,Teresa LeitÃ£o,Maria Teresa LeitÃ£o de Melo,Feminino,Senadora,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador6338.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/6338,sen.teresaleitao@senado.leg.br,33032423,PT,PE,NaN,NÃ£o,Sim,335,Bloco Parlamentar da ResistÃªncia DemocrÃ¡tica,BLPRD,2023-02-01,NaN
8,4560,850,SÃ©rgio PetecÃ£o,SÃ©rgio de Oliveira Cunha,Masculino,Senador,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador4560.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/4560,sen.sergiopetecao@senado.leg.br,33034086,PSD,AC,NaN,NÃ£o,NÃ£o,335,Bloco Parlamentar da ResistÃªncia DemocrÃ¡tica,BLPRD,2023-02-01,http://www.petecao.com.br/
9,5959,817,Styvenson Valentim,Eann Styvenson Valentim Mendes,Masculino,Senador,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador5959.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/5959,sen.styvensonvalentim@senado.leg.br,33031148,PODEMOS,RN,NaN,Sim,Sim,336,Bloco Parlamentar Democracia,BLDEM,2023-02-02,NaN


In [55]:
dados.to_csv("../Dados/Senadores.csv",encoding='utf-8-sig')